<a href="https://colab.research.google.com/github/Gabriela-Alcaide/An-lise-de-Redes-Sociais/blob/main/Extra%C3%A7%C3%A3o_de_dados_Bluesky.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import base64
import re
import time
import random
import json
import csv

In [ ]:
import pandas as pd
from google.colab import files

In [ ]:
usuario = 'xx' # E-mail cadastrado na API - dado confidencial.
senha = 'xx' # Senha cadastrada na API - dado confidencial.
url_criar_sessao = 'https://bsky.social/xrpc/com.atproto.server.createSession'

corpo_req = {
    "identifier": usuario,
    "password": senha,
}

req = requests.post(url_criar_sessao, json=corpo_req)

token = req.json()['accessJwt']

In [ ]:
headers = {
    'Authorization': f'Bearer {token}'
}

In [ ]:
usuarios_handles = []

palavras_possiveis = ['eleições', 'São Paulo', 'Boulos', 'Ricardo Nunes', 'Tabata', 'Datena', 'Marçal']

while len(usuarios_handles) < 200:
  palavras = random.sample(palavras_possiveis, 3)
  palavras = ','.join(palavras)
  url_get_posts = f'https://bsky.social/xrpc/app.bsky.feed.searchPosts?q={palavras}'
  req = requests.get(url_get_posts, headers=headers)
  for post in req.json()['posts']:
    if post['author']['handle'] not in usuarios_handles:
      usuarios_handles.append(post['author']['handle'])
  time.sleep(2)

KeyboardInterrupt: 

In [ ]:
len(usuarios_handles)

203

In [ ]:
import threading
import requests

In [ ]:
url_get_posts = f'https://bsky.social/xrpc/app.bsky.feed.searchPosts'
usuarios_posts = []
lock = threading.Lock()

def get_posts_usuario(usuario):
  params = {
    'q': '*',
    'author': usuario
  }
  retries = 5
  for attempt in range(retries):
    try:
      req = requests.get(url_get_posts, params = params, headers=headers)
      req.raise_for_status()
      with lock:
        usuarios_posts.append(req.json())
      break
    except requests.exceptions.RequestException as e:
      if attempt < retries - 1:
        time.sleep(2 ** attempt)

threads = []
for u in usuarios_handles:
  t = threading.Thread(target=get_posts_usuario, args=(u,))
  threads.append(t)
  t.start()

for t in threads:
  t.join()

In [ ]:
usuarios_posts_tratados = []
expressao_regular_data = r'(\d{4}-\d{2}-\d{2})'

for u in range(len(usuarios_posts)):
  posts = []
  for p in usuarios_posts[u]['posts']:
    post = {
      'handle_author': p['author']['handle'],
      'conteudo': p['record']['text'],
      'num_respostas': p['replyCount'],
      'num_repostagens': p['repostCount'],
      'num_likes': p['likeCount'],
      'num_citacoes': p['quoteCount']
    }
    posts.append(post)
  usuarios_posts_tratados.append(posts)

In [ ]:
url_obter_perfis = 'https://bsky.social/xrpc/app.bsky.actor.getProfiles'

num_grupos = (len(usuarios_handles) // 25) + 1

grupos = [f"Grupo_{n}" for n in range(num_grupos)]

grupo_olhando_agora = 0

grupos_25_usuarios = {}

for n in range(num_grupos):
  grupos_25_usuarios[f"Grupo_{n}"] = usuarios_handles[n*25:(n+1)*25]

headers = {
    'Authorization': f'Bearer {token}'
}

usuarios = []

for grupo in grupos_25_usuarios:
  grupo = grupos_25_usuarios[grupo]
  handles_str = '&'.join([f'actors={handle}' for handle in grupo])
  url = f'{url_obter_perfis}?{handles_str}'

  req = requests.get(url, headers=headers)
  usuarios = usuarios + req.json()['profiles']

In [ ]:
usuarios_tratados = []
expressao_regular_data = r'(\d{4}-\d{2}-\d{2})'

for u in usuarios:
  usuarios_tratados.append({
      'handle': u['handle'],
      'did': u['did'],
      'displayName': f"{u['displayName'] if 'displayName' in u else 'None'}",
      'imagem_perfil': f"{u['avatar'] if 'avatar' in u else 'None'}",
      'imagem_banner': f"{u['banner'] if 'banner' in u else 'None'}",
      'data_criacao': re.search(expressao_regular_data, u['createdAt']).group(1),
      'seguidores': u['followersCount'],
      'seguindo': u['followsCount'],
      'numPosts': u['postsCount']
  })

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
analisador_sentimentos = pipeline("sentiment-analysis") # Sentimentos possíveis: Positive, Negative e Neutral.

from nltk import FreqDist
from nltk import word_tokenize

!pip install spacy
!python -m spacy download pt_core_news_sm
import spacy
nlp = spacy.load("pt_core_news_sm")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 10.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
threads = []
lock2 = threading.Lock()

def adicionar_infos_posts(num_usuario):

  for p in usuarios_posts_tratados[num_usuario]:
    with lock2:
      conteudo = p['conteudo']

    sentimento = analisador_sentimentos(conteudo)[0]['label']

    doc = nlp(conteudo)

    verbos = 0
    substantivos_comuns = 0
    substantivos_proprios = 0
    adjetivos = 0

    for token in doc:
      if token.is_alpha and not token.is_stop:
        if token.pos_ == 'PROPN':
          substantivos_proprios += 1
        elif token.pos_ == 'NOUN':
          substantivos_comuns += 1
        elif token.pos_ == 'VERB':
          verbos += 1
        elif token.pos_ == 'ADJ':
          adjetivos += 1

    with lock2:
      p['sentimento'] = sentimento
      p['substantivos_proprios'] = substantivos_proprios
      p['substantivos_comuns'] = substantivos_comuns
      p['verbos'] = verbos
      p['adjetivos'] = adjetivos

for u in range(len(usuarios_handles)):
  t = threading.Thread(target=adicionar_infos_posts, args=(u,))
  threads.append(t)
  t.start()

for t in threads:
  t.join()

In [ ]:
from collections import Counter

In [ ]:
threads = []
lock3 = threading.Lock()

def gerar_infos(num_usuario):
  posts_desse_usuario = usuarios_posts_tratados[num_usuario]

  sentimentos_nos_posts = []
  quantidade_substantivos_proprios = 0
  quantidade_substantivos_comuns = 0
  quantidade_verbos = 0
  quantidade_adjetivos = 0
  total_respostas = 0
  total_repostagens = 0
  total_curtidas = 0
  total_citacoes = 0

  for p in posts_desse_usuario:
    total_respostas = total_respostas + p['num_respostas']
    total_repostagens = total_repostagens + p['num_repostagens']
    total_curtidas = total_curtidas + p['num_likes']
    total_citacoes = total_citacoes + p['num_citacoes']
    sentimentos_nos_posts.append(p['sentimento'])
    quantidade_substantivos_proprios = quantidade_substantivos_proprios + p['substantivos_proprios']
    quantidade_substantivos_comuns = quantidade_substantivos_comuns + p['substantivos_comuns']
    quantidade_verbos = quantidade_verbos + p['verbos']
    quantidade_adjetivos = quantidade_adjetivos + p['adjetivos']

  contador = Counter(sentimentos_nos_posts)
  sentimento_mais_frequente, frequencia = contador.most_common(1)[0]

  palavras_importantes = quantidade_substantivos_proprios + quantidade_substantivos_comuns + quantidade_verbos + quantidade_adjetivos

  handle = usuarios_posts_tratados[num_usuario][0]['handle_author']

  for u in usuarios_tratados:
    if u['handle'] == handle:
      with lock3:
        u['media_respostas'] = total_respostas / u["numPosts"]
        u['media_repostagens'] = total_repostagens / u["numPosts"]
        u['media_curtidas'] = total_curtidas / u["numPosts"]
        u['media_citacoes'] = total_citacoes / u["numPosts"]
        u['sentimento_mais_frequente'] = sentimento_mais_frequente
        u['porc_substantivos_proprios'] = quantidade_substantivos_proprios / palavras_importantes
        u['porc_substantivos_comuns'] = quantidade_substantivos_comuns / palavras_importantes
        u['porc_verbos'] = quantidade_verbos / palavras_importantes
        u['porc_adjetivos'] = quantidade_adjetivos / palavras_importantes

for u in range(len(usuarios_handles)):
  t = threading.Thread(target=gerar_infos, args=(u,))
  threads.append(t)
  t.start()

for t in threads:
  t.join()

In [ ]:
import json

with open('bluesky2.json', 'w', encoding='utf-8') as f:
  json.dump(usuarios_tratados, f, ensure_ascii=False, indent=4)

In [ ]:
files.download('bluesky2.json')